# Práctica 3 Reconocimiento de Formas: Clasificador Estadístico Bayesiano

* **Alumno 1**: Javier Barragán Haro
* **Alumno 2**: Victor Nieves Sanchez


In [1]:
import numpy as np
from abc import abstractmethod
import pandas as pd


class Classifier:

    @abstractmethod
    def fit(self,X,y):
        pass

    @abstractmethod
    def predict(self,X):
        pass

class ClassifBayesiano(Classifier):
    def __init__(self, labels=[]):
        """Constructor de la clase
        labels: lista de etiquetas de esta clase"""
        self.labels = labels
        self.ln_apriories = None
        self.means = None
        self.ln_determinants = None
        self.inv_covs = None
        
    def fit(self,X,y):
        """Entrena el clasificador. Dado que es un clasificador Gausiano Bayesiano, 
        se aprenderán los parámetros de las gausianas de cada clase.
        X: matriz numpy cada fila es un dato, cada columna una característica
        y: vector de etiquetas, tantos elementos como filas en X
        retorna objeto clasificador"""
        assert X.ndim == 2 and X.shape[0] == len(y)
        
        y = pd.factorize(y)[0]
        
        # Contar cuantos ejemplos hay de cada etiqueta
        unique, counts = np.unique(y, return_counts=True)
        
        # Usando el contador de ejemplos de cada etiqueta, calcular el logaritmo neperiano de las probabilidades a-priori
        self.ln_apriories = np.array([np.log((counts[i]/np.sum(counts))) for i in unique])
        
        # Calcular para los ejemplos de cada clase, la media de cada una de sus características (centroide)
        self.means = np.array([np.mean(X[y==i], axis=0) for i in unique])
        
        # Sustraer a los ejemplos de cada clase su media y calcular su matriz de covarianzas (puedes emplear compresión de listas) 
        covs = np.array([np.cov(X[y==i] - self.means[i], rowvar=False) for i in unique])

        # Para cada una de las clases, calcular el logaritmo neperiano de su matriz de covarianzas (puedes emplear compresión de listas o la función map)
        self.ln_determinants = np.log(list(map(np.linalg.det, covs)))
        
        # Para cada una de las clases, calcular la inversa de su matriz de covarianzas (puedes emplear compresión de listas o la función map)
        self.inv_covs = np.array(list(map(np.linalg.pinv,covs)))

        return self

    def predict(self,X):
        """Estima el grado de pertenencia de cada dato a todas las clases 
        X: matriz numpy cada fila es un dato, cada columna una medida del vector de caracteristicas. 
        Retorna una matriz, con tantas filas como datos y tantas columnas como clases tenga
        el problema, cada fila almacena los valores pertenencia de un dato a cada clase""" 
        assert self.means is not None, "Error: The classifier needs to be fitted. Please call fit(X, y) method."
        assert X.ndim == 2 and X.shape[1] == self.means.shape[1]

        # Resta la media de cada clase a cada ejemplo en X
        X_mean0 = self.means[:,np.newaxis,:] - X
        
        # Calcula el logaritmo de la función de decisión gausiana 
        # Transparencias de Métodos paramétricos de clasificación: página 14:
        # -(1/2)ln|Sigma_i| - (1/2)*(x- mu_i)^T Sigma_i^-1 (x- mu_i) + lnP(alpha_i)
        return -0.5 * self.ln_determinants[:, np.newaxis] \
            - 0.5 * np.array([np.sum((X_mean0[i] @ self.inv_covs[i]) 
                                     * X_mean0[i], axis=1) for i in np.unique(y)]) \
            + self.ln_apriories[:, np.newaxis]

        
    def pred_label(self,X):
        """Estima la etiqueta de cada dato. La etiqueta puede ser un entero o bien un string.
        X: matriz numpy cada fila es un dato, cada columna una medida
        retorna un vector con las etiquetas de cada dato"""
        return np.argmax(X, axis = 0)
    
    def num_aciertos(self, X): 
        """Cuenta el numero de aciertos del clasificador para un conjunto de datos X.
        X: matriz de datos a clasificar"""
        same_values = []
        [same_values.append(X[i] == self.labels[i]) for i in range(0, len(self.labels))]
        number = same_values.count(True)

        return number, (number / len(X)) * 100

## Iris Dataset
Carga, entrenamiento, predicción y evaluación en la base de datos de Iris:

In [2]:
# 1. Cargar los datos de la base de datos de entrenamiento
from sklearn.datasets import load_iris
dataset = load_iris()
X = dataset.data
# print("X: \n" + str(X))
y = dataset.target
# print("y: \n" + str(y))

# 2. Entrenar el clasificador
ClassifBayes = ClassifBayesiano(y)
ClassifBayes.fit(X,y)

# 3. Predecir empleando la base de datos de entrenamiento (X)
predict_matrix = ClassifBayes.predict(X)

# 4. Evaluar el clasificador calculando el porcentaje de datos correctamente clasificados
labels_matrix = ClassifBayes.pred_label(predict_matrix)

correct = ClassifBayes.num_aciertos(labels_matrix)
print("Correct answers:", correct[0], "/", len(y))
print("Success rate:", correct[1])

Correct answers: 147 / 150
Success rate: 98.0


## Wine dataset
Carga, entrenamiento, predicción y evaluación en la base de datos de Wine:

In [3]:
# 1. Cargar los datos de la base de datos de entrenamiento
from sklearn.datasets import load_wine
dataset = load_wine()
X = dataset.data
# print("X: \n" + str(X))
y = dataset.target
# print("y: \n" + str(y))

# 2. Entrenar el clasificador
ClassifBayes = ClassifBayesiano(y)
ClassifBayes.fit(X,y)

# 3. Predecir empleando la base de datos de entrenamiento (X)
predict_matrix = ClassifBayes.predict(X)

# 4. Evaluar el clasificador calculando el porcentaje de datos correctamente clasificados
labels_matrix = ClassifBayes.pred_label(predict_matrix)

correct = ClassifBayes.num_aciertos(labels_matrix)
print("Correct answers:", correct[0], "/", len(y))
print("Success rate:", correct[1])

Correct answers: 177 / 178
Success rate: 99.43820224719101


## Breast cancer dataset
Carga, entrenamiento, predicción y evaluación en la base de datos de Breast cancer:

In [4]:
# 1. Cargar los datos de la base de datos de entrenamiento
from sklearn.datasets import load_breast_cancer
dataset = load_breast_cancer()
X = dataset.data
# print("X: \n" + str(X))
y = dataset.target
# print("y: \n" + str(y))

# 2. Entrenar el clasificador
ClassifBayes = ClassifBayesiano(y)
ClassifBayes.fit(X,y)

# 3. Predecir empleando la base de datos de entrenamiento (X)
predict_matrix = ClassifBayes.predict(X)

# 4. Evaluar el clasificador calculando el porcentaje de datos correctamente clasificados
labels_matrix = ClassifBayes.pred_label(predict_matrix)

correct = ClassifBayes.num_aciertos(labels_matrix)
print("Correct answers:", correct[0], "/", len(y))
print("Success rate:", correct[1])

Correct answers: 554 / 569
Success rate: 97.36379613356766


## MNIST Database (Modified National Institute of Standards and Technology database)
MNIST es una base de datos de texto manuscrito, que se usa de forma clásica para entrenar sistemas de procesado de imágenes.

Carga, entrenamiento, predicción y evaluación en la base de datos de MNIST:

In [5]:
from sklearn.datasets import fetch_openml

# Cargamos MNIST desde internet ( https://www.openml.org/d/554 )
all_X, all_y = fetch_openml('mnist_784', version=1, return_X_y=True, cache=True)
X = all_X[:60000]
y = all_y[:60000]
y = pd.factorize(y)[0]

# print("X: \n" + str(X))
# print("y: \n" + str(y))

# 2. Entrenar el clasificador
ClassifBayes = ClassifBayesiano(y)
ClassifBayes.fit(X,y)

# 3. Predecir empleando la base de datos de entrenamiento (X)
predict_matrix = ClassifBayes.predict(X)

# 4. Evaluar el clasificador calculando el porcentaje de datos correctamente clasificados
labels_matrix = ClassifBayes.pred_label(predict_matrix)

correct = ClassifBayes.num_aciertos(labels_matrix)
print("Correct answers:", correct[0], "/", len(y))
print("Success rate:", correct[1])

/usr/lib/python3/dist-packages/ipykernel_launcher.py:49: RuntimeWarning: divide by zero encountered in log


Correct answers: 5421 / 60000
Success rate: 9.035


## Isolet Dataset (Isolated Letter Speech Recognition)
Carga, entrenamiento, predicción y evaluación en la base de datos de Isolet

In [6]:
import pandas as pd

# 1. Cargar los datos de la base de datos de entrenamiento
X, y = fetch_openml('isolet', version=1, return_X_y=True, cache=True)
y = pd.factorize(y)[0]
print("X: \n" + str(X))
print("y: \n" + str(y))

# Nota: Si la matriz de covarianzas no tiene rango completo, no podrás invertirlas, 
# 2. Entrenar el clasificador
ClassifBayes = ClassifBayesiano(y)
ClassifBayes.fit(X,y)

# 3. Predecir empleando la base de datos de entrenamiento (X)
predict_matrix = ClassifBayes.predict(X)

# 4. Evaluar el clasificador calculando el porcentaje de datos correctamente clasificados
labels_matrix = ClassifBayes.pred_label(predict_matrix)

correct = ClassifBayes.num_aciertos(labels_matrix)
print("Correct answers:", correct[0], "/", len(y))
print("Success rate:", correct[1])

X: 
[[-0.4394 -0.093   0.1718 ...  0.641   0.5898 -0.4872]
 [-0.4348 -0.1198  0.2474 ...  0.4318  0.4546 -0.091 ]
 [-0.233   0.2124  0.5014 ...  0.254   0.1588 -0.4762]
 ...
 [-0.6696 -0.373   0.1584 ...  0.0728  0.0728 -0.5818]
 [-0.5764 -0.1764  0.5106 ...  0.3044 -0.0434 -0.5   ]
 [-0.6624 -0.3334  0.3666 ... -0.0894 -0.1708 -0.317 ]]
y: 
[ 0  0  1 ... 24 25 25]


/usr/lib/python3/dist-packages/ipykernel_launcher.py:49: RuntimeWarning: divide by zero encountered in log


Correct answers: 300 / 7797
Success rate: 3.8476337052712584


## Resultados
Resultados de los tres experimentos:

| Base de datos | Número de aciertos | Porcentaje de aciertos |
| --- | --- | --- |
| Iris   | 147 | 98.00 |
| Wine   | 177 | 99.43 |
| Cancer | 554 | 97.36 |
| MNIST  | 5421 / 60000 | 9.03 |
| Isolet | 300 / 7797 | 3.85 |